## HPO Raw Data EDA

In [1]:
import pandas as pd

In [2]:
!wget -P /tmp http://compbio.charite.de/jenkins/job/hpo.annotations/lastSuccessfulBuild/artifact/util/annotation/genes_to_phenotype.txt

--2020-06-17 19:36:59--  http://compbio.charite.de/jenkins/job/hpo.annotations/lastSuccessfulBuild/artifact/util/annotation/genes_to_phenotype.txt
Resolving compbio.charite.de (compbio.charite.de)... 141.42.207.15
Connecting to compbio.charite.de (compbio.charite.de)|141.42.207.15|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14433494 (14M) [text/plain]
Saving to: ‘/tmp/genes_to_phenotype.txt’

genes_to_phenotype. 100%[===================>]  13.76M  2.87MB/s    in 8.3s    

2020-06-17 19:37:08 (1.67 MB/s) - ‘/tmp/genes_to_phenotype.txt’ saved [14433494/14433494]



In [3]:
path = '/tmp/genes_to_phenotype.txt'

In [16]:
!head -n 2 $path

#Format: entrez-gene-id<tab>entrez-gene-symbol<tab>HPO-Term-Name<tab>HPO-Term-ID<tab>Frequency-Raw<tab>Frequency-HPO<tab>Additional Info from G-D source<tab>G-D source<tab>disease-ID for link
8192	CLPP	HP:0000786	Primary amenorrhea			-	mim2gene	OMIM:614129


In [59]:
cols

['entrez-gene-id',
 'entrez-gene-symbol',
 'HPO-Term-Name',
 'HPO-Term-ID',
 'Frequency-Raw',
 'Frequency-HPO',
 'Additional Info from G-D source',
 'G-D source',
 'disease-ID for link']

In [24]:
cols = !head -n 1 $path | cut -d' ' -f 2- |  sed -e 's/<tab>/,/g'
cols = cols[0].split(',')
df = pd.read_csv(path, sep='\t', names=cols, skiprows=1, header=None)
df.head()

,entrez-gene-id,entrez-gene-symbol,HPO-Term-Name,HPO-Term-ID,Frequency-Raw,Frequency-HPO,Additional Info from G-D source,G-D source,disease-ID for link
0,8192,CLPP,HP:0000786,Primary amenorrhea,NaN,NaN,-,mim2gene,OMIM:614129
1,8192,CLPP,HP:0000013,Hypoplasia of the uterus,NaN,NaN,-,mim2gene,OMIM:614129
2,8192,CLPP,HP:0001250,Seizure,NaN,HP:0040283,-,mim2gene,OMIM:614129
3,8192,CLPP,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:614129
4,8192,CLPP,HP:0004322,Short stature,NaN,HP:0040283,-,mim2gene,OMIM:614129


In [58]:
# ALL HPO terms are simply repeated for each gene that is associated with a disease
(
    df
    .groupby(['disease-ID for link', 'entrez-gene-id'])
    .apply(lambda g: ','.join(list(sorted(g['HPO-Term-Name'].unique()))))
    .rename('terms')
    .reset_index()
    .groupby('disease-ID for link')['terms'].nunique().value_counts()
)

1    7317
Name: terms, dtype: int64

### Terms related to inheritance mode

In [62]:
df[df['HPO-Term-ID'].str.lower().str.contains('inheritance')]['HPO-Term-ID'].value_counts()

Autosomal recessive inheritance                            3063
Autosomal dominant inheritance                             2551
X-linked recessive inheritance                              230
X-linked dominant inheritance                                75
X-linked inheritance                                         72
Mitochondrial inheritance                                    56
Multifactorial inheritance                                   39
Polygenic inheritance                                        32
Y-linked inheritance                                         18
Digenic inheritance                                           8
Autosomal dominant inheritance with maternal imprinting       2
Oligogenic inheritance                                        1
Name: HPO-Term-ID, dtype: int64

In [28]:
dff = df[
    df['HPO-Term-ID'].str.lower().str.contains('dominant') |
    df['HPO-Term-ID'].str.lower().str.contains('recessive')
]
dff.head()

,entrez-gene-id,entrez-gene-symbol,HPO-Term-Name,HPO-Term-ID,Frequency-Raw,Frequency-HPO,Additional Info from G-D source,G-D source,disease-ID for link
3,8192,CLPP,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:614129
8,2,A2M,HP:0000006,Autosomal dominant inheritance,NaN,NaN,susceptibility,mim2gene,OMIM:104300
59,8195,MKKS,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:236700
69,8195,MKKS,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:605231
140,8200,GDF5,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:113100


### Inheritance mode count by gene

In [40]:
cts = dff.groupby('entrez-gene-id')['HPO-Term-Name'].nunique()
cts.value_counts()

1    3250
2     610
3      17
Name: HPO-Term-Name, dtype: int64

In [41]:
cts.sort_values().tail()

entrez-gene-id
5148     3
23746    3
2516     3
1259     3
1406     3
Name: HPO-Term-Name, dtype: int64

In [34]:
cts[cts == 2].head()

entrez-gene-id
16     2
19     2
58     2
60     2
103    2
Name: HPO-Term-Name, dtype: int64

In [35]:
dff[dff['entrez-gene-id'] == 16]

,entrez-gene-id,entrez-gene-symbol,HPO-Term-Name,HPO-Term-ID,Frequency-Raw,Frequency-HPO,Additional Info from G-D source,G-D source,disease-ID for link
425,16,AARS1,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:616339
429,16,AARS1,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:613287


In [36]:
dff[dff['entrez-gene-id'] == 19]

,entrez-gene-id,entrez-gene-symbol,HPO-Term-Name,HPO-Term-ID,Frequency-Raw,Frequency-HPO,Additional Info from G-D source,G-D source,disease-ID for link
462,19,ABCA1,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:205400
478,19,ABCA1,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:604091


In [33]:
dff[dff['entrez-gene-id'] == 5148]

,entrez-gene-id,entrez-gene-symbol,HPO-Term-Name,HPO-Term-ID,Frequency-Raw,Frequency-HPO,Additional Info from G-D source,G-D source,disease-ID for link
128908,5148,PDE6G,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:268000
128910,5148,PDE6G,HP:0001419,X-linked recessive inheritance,NaN,NaN,-,mim2gene,OMIM:268000
128912,5148,PDE6G,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:268000
128916,5148,PDE6G,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:613582


### Inheritance mode count by disease

In [37]:
cts = dff.groupby('disease-ID for link')['HPO-Term-Name'].nunique()
cts.value_counts()

1    4762
2     239
3       3
Name: HPO-Term-Name, dtype: int64

In [39]:
cts[cts == 2].head()

disease-ID for link
OMIM:105400    2
OMIM:106100    2
OMIM:107600    2
OMIM:113750    2
OMIM:113800    2
Name: HPO-Term-Name, dtype: int64

Diseases with multiple genes and different inheritance patterns amongst those genes:

In [46]:
cts = dff.groupby('disease-ID for link')['entrez-gene-id'].nunique()
diseases = cts[cts > 1].index.values
cts = dff.pipe(lambda df: df[df['disease-ID for link'].isin(diseases)]).groupby('disease-ID for link')['HPO-Term-Name'].nunique()
cts.sort_values().tail()

disease-ID for link
OMIM:149400    2
OMIM:240500    2
OMIM:606176    2
OMIM:176100    2
OMIM:268000    3
Name: HPO-Term-Name, dtype: int64

In [48]:
dff.pipe(lambda df: df[df['disease-ID for link'] == 'OMIM:606176'])

,entrez-gene-id,entrez-gene-symbol,HPO-Term-Name,HPO-Term-ID,Frequency-Raw,Frequency-HPO,Additional Info from G-D source,G-D source,disease-ID for link
82757,2645,GCK,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:606176
82771,2645,GCK,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:606176
101128,3630,INS,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:606176
101142,3630,INS,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:606176
101557,3651,PDX1,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:606176
101571,3651,PDX1,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:606176
104048,3767,KCNJ11,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:606176
104062,3767,KCNJ11,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:606176
179123,6833,ABCC8,HP:0000006,Autosomal dominant inheritance,NaN,NaN,-,mim2gene,OMIM:606176
179137,6833,ABCC8,HP:0000007,Autosomal recessive inheritance,NaN,NaN,-,mim2gene,OMIM:606176
